In [ ]:
import os
import cv2

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import pickle

import tensorflow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.metrics import Precision, Recall, FalseNegatives, FalsePositives, TruePositives, TrueNegatives

from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2S

In [ ]:
os.environ['KAGGLE_USERNAME'] = "lorddexter"
os.environ['KAGGLE_KEY'] = "bf0fec1866a689351f73244d4dd09a99"
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria

 98% 665M/675M [00:19<00:00, 23.8MB/s]
100% 675M/675M [00:19<00:00, 35.6MB/s]


In [ ]:
!unzip cell-images-for-detecting-malaria.zip

Streaming output truncated to the last 5000 lines.
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_116.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_118.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_126.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_134.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_141.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_168.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_175.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_183.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_221.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_222.png  
  inflating: cell_i

In [ ]:
base_path = '/content/cell_images/cell_images'

In [ ]:
def load_and_preprocess_images(base_path):

  Image_files = []
  Labels = []
  Dataset = []
  
  for label in os.listdir(base_path):

    # append labels from sub-directory names
    Labels.append(label)
    
    # creating image path
    images_path = os.path.join(base_path, label)
    
    for image_name in os.listdir(images_path):

      image_paths = os.path.join(images_path, image_name)
      
      # read the images
      img = cv2.imread(image_paths)

      try:
        
        # resizing the images
        img = cv2.resize(img, (64, 64), cv2.INTER_CUBIC)

        # make copy of resized image for edge detection
        img_copy = img.copy()
    
        # convert the images to grayscale for edge detection
        img_copy = cv2.cvtColor(img_copy, cv2.COLOR_BGR2RGB)   
        img_copy = cv2.cvtColor(img_copy, cv2.COLOR_RGB2GRAY)

        # preprocessing using canny edge detector to make image edges sharper and image smoother
        img_copy = cv2.GaussianBlur(img, (7,7), 0) 
        edges = cv2.Canny(img_copy, threshold1 = 80, threshold2 = 160)
        edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
        
        # superimposing image with detected edges
        final_img = cv2.addWeighted(img, 0.5, edges, 0.5, 0)
        
        Dataset.append([final_img, np.array(label)])
      except:
        continue

  return Dataset

In [ ]:
Dataset = load_and_preprocess_images(base_path)

In [ ]:
Data = pd.DataFrame(Dataset, columns = ['Images', 'Labels'])

In [ ]:
Data.head()

,Images,Labels
0,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",Uninfected
1,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",Uninfected
2,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",Uninfected
3,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",Uninfected
4,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",Uninfected


In [ ]:
def Labels_to_numeric(x):
        if x == 'Parasitized':
          return 1
        if x == 'Uninfected':
          return 0

Data['Labels'] = Data['Labels'].apply(Labels_to_numeric)
print(Data.head())

                                              Images  Labels
0  [[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...       0
1  [[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...       0
2  [[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...       0
3  [[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...       0
4  [[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...       0


In [ ]:
x = np.array([i for i in Data['Images']]).reshape(-1, 64, 64, 3)
y = to_categorical([i for i in Data['Labels']])

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(x, y, test_size=0.2, shuffle = True, random_state = 2)

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(Y_train.shape)
print(Y_val.shape)

(22046, 64, 64, 3)
(5512, 64, 64, 3)
(22046, 2)
(5512, 2)


In [ ]:
X_train = X_train/255.
X_val = X_val/255.

In [ ]:
# defining evaluation metrics

precision = Precision()
recall = Recall()
false_positive = FalsePositives()
false_negative = FalseNegatives()
true_positive = TruePositives()
true_negative = TrueNegatives()

In [ ]:
class myCallback(tensorflow.keras.callbacks.Callback):

  def on_epoch_end(self, epochs, logs = {}):
    if (logs.get('accuracy') > 0.98):
      self.model.stop_training = True

myCallback = myCallback()

In [ ]:
efficient_net_v2 = EfficientNetV2S(include_top = False, input_shape = (64, 64, 3), weights = 'imagenet')
efficient_net_v2.trainable = False

82436096/82420632 [==============================] - 1s 0us/step


In [ ]:
x = Flatten()(efficient_net_v2.output)
dense1 = Dense(units = 1012, activation = 'relu')(x)
output = Dense(units=2, activation='sigmoid')(dense1)
efficient_net_model = Model(inputs = efficient_net_v2.input, outputs = output)

In [ ]:
efficient_net_v2_adam = Adam(learning_rate = 0.0001)
efficient_net_v2_sgd = SGD(learning_rate = 0.0001, momentum = 0.9, nesterov = True)

efficient_net_model.compile(optimizer = efficient_net_v2_sgd, loss = 'binary_crossentropy', metrics = ['accuracy', precision, recall, true_positive, true_negative, false_positive, false_negative])

In [ ]:
efficient_net_model.fit(x = X_train, y = Y_train, validation_data = [X_val, Y_val], epochs = 30, callbacks = [myCallback])

Epoch 1/30
689/689 [==============================] - 51s 43ms/step - loss: 0.5990 - accuracy: 0.7126 - precision: 0.7140 - recall: 0.6987 - true_positives: 15403.0000 - true_negatives: 15876.0000 - false_positives: 6170.0000 - false_negatives: 6643.0000 - val_loss: 0.5692 - val_accuracy: 0.7215 - val_precision: 0.7217 - val_recall: 0.7164 - val_true_positives: 3949.0000 - val_true_negatives: 3989.0000 - val_false_positives: 1523.0000 - val_false_negatives: 1563.0000
Epoch 2/30
689/689 [==============================] - 25s 37ms/step - loss: 0.5478 - accuracy: 0.7335 - precision: 0.7351 - recall: 0.7297 - true_positives: 16088.0000 - true_negatives: 16248.0000 - false_positives: 5798.0000 - false_negatives: 5958.0000 - val_loss: 0.5491 - val_accuracy: 0.7297 - val_precision: 0.7306 - val_recall: 0.7244 - val_true_positives: 3993.0000 - val_true_negatives: 4040.0000 - val_false_positives: 1472.0000 - val_false_negatives: 1519.0000
Epoch 3/30
689/689 [==============================] - 33

In [ ]:
efficient_net_model.save(filepath = 'efficient_net_v2_model', save_format = 'h5', include_optimizer = True)

In [ ]:
efficient_net_model.save(filepath = 'efficient_net_v2_model_tf', save_format = 'tf', include_optimizer = True)

INFO:tensorflow:Assets written to: inception_model_tf/assets


In [ ]:
pickle.dump(efficient_net_model, open('efficient_net_v2_model.pkl', 'wb'))

INFO:tensorflow:Assets written to: ram://17235dab-da91-4a7f-bdca-5c072823f829/assets
